<a href="https://colab.research.google.com/github/JHyunjun/TF2.0_2D-CNN/blob/main/TimeSeries_2DCNN_Ensemble_AE_withAGWN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.04.29

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, GRU, Dropout, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model


xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_train = np.delete(xy_train ,3, axis = 1)
xy_test = np.delete(xy_test ,3, axis = 1)

start_test = 1000
middle_test = 2000

middle_test1 = 3000
end_test = 10000

xy_test_1 = xy_test[:start_test,:]
xy_test_1n2 = xy_test[start_test:middle_test,:]
xy_test_2 = xy_test[middle_test1:end_test,:]

xy_train = np.concatenate((xy_train, xy_test_1, xy_test_2), axis = 0)
xy_test = np.copy(xy_train) # For Adding AWGN
#xy_test = xy_test_1n2

print("Train : ", xy_train.shape)
print("Test : " , xy_test.shape)


def add_awgn_noise(signal, snr_dB):
    snr_linear = 10**(snr_dB / 10)
    signal_power = np.mean(signal**2)
    noise_power = signal_power / snr_linear
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    noisy_signal = signal + noise
    
    return noisy_signal

snr_dB = 40
number_of_AWGN = 20

# Add AWGN with 30 dB SNR to the original data
noisy_xy_train = add_awgn_noise(xy_train, snr_dB)

all_noisy_data = [noisy_xy_train]
for _ in range(number_of_AWGN):
    noisy_xy_train = add_awgn_noise(xy_train, snr_dB)
    all_noisy_data.append(noisy_xy_train)

final_dataset = np.concatenate(all_noisy_data, axis=0)

# Check the final dataset shape
print(final_dataset.shape)
xy_train = np.copy(final_dataset)


In [ ]:
#Normalization

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)

In [ ]:
def add_awgn_noise(signal, snr):
    signal = np.array(signal)
    signal_power = np.mean(np.square(signal))
    noise_power = signal_power / (10 ** (snr / 10))
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    return signal + noise

#signal_value = np.ones(4)
signal_value = [-13.700012,  25,       100,       746.3] 
snr_values = [30, 30, 30, 30, 30, 30, 40]

print("Original Data : ", signal_value)
for snr in snr_values:
    noisy_value = add_awgn_noise(signal_value, snr)
    print(f"SNR {snr} dB에 대한 값: {noisy_value}")

In [ ]:
# Noise Simulation
def simulate_awgn(signal, snr, num_trials=10):
    noisy_signals = []
    for _ in range(num_trials):
        noisy_signal = add_awgn_noise(signal, snr)
        noisy_signals.append(noisy_signal)
    return np.array(noisy_signals)

original_data = np.array([-13.700012,  25,       100,       746.3])
snr_values = [5, 10, 15, 20, 25, 30, 40]

print("Original Data : ", original_data)
for snr in snr_values:
    simulated_data = simulate_awgn(original_data, snr)
    mean_data = np.mean(simulated_data, axis=0)
    std_data = np.std(simulated_data, axis=0)
    print(f"SNR {snr} dB에 대한 평균 값: {mean_data}")
    print(f"SNR {snr} dB에 대한 표준 편차: {std_data}")

In [ ]:
print(xy_train.shape)
print(xy_test.shape)

In [ ]:
def reshape_timeseries_to_image(data, window_size):
    num_features = data.shape[1]
    num_samples = data.shape[0] - window_size + 1
    reshaped_data = np.zeros((num_samples, window_size, num_features))

    for i in range(num_samples):
        reshaped_data[i] = data[i:i+window_size]

    return reshaped_data

In [ ]:
TIME_STEPS = 6

train_image = reshape_timeseries_to_image(xy_train, TIME_STEPS)
test_image = reshape_timeseries_to_image(xy_test, TIME_STEPS)
print("Train image.shape : ", train_image.shape,train_image[0,:,:])
print("Test image.shape : " ,test_image.shape, test_image[0,:,:])

In [ ]:
input_shape = (TIME_STEPS, train_image.shape[2], 1)
inputs = Input(shape=input_shape)

Base_Filter = 128
Decoder_count = 1

# Encoding
def create_encoder() : 
    inputs = tf.keras.Input(shape = input_shape)
    x = Conv2D(Base_Filter, (3, 3), activation='relu', padding='same')(inputs)
    #x = BatchNormalization()(x)
    x = Conv2D(Base_Filter/2, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(Base_Filter/4, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 1), padding='same')(x)

    return encoded, inputs

# Decoding
def create_ensemble(encoded, inputs):
    ensemble_outputs = []
    
    for i in range(Decoder_count) : 
        encoded, inputs = create_encoder() # Encoder-Decoder Integrated Ensemble
        x = Conv2D(Base_Filter/4, (3, 3), activation='relu', padding='same')(encoded)
        #x = BatchNormalization()(x)
        x = Conv2D(Base_Filter/4, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(Base_Filter/2, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(Base_Filter, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 1))(x)
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        model = tf.keras.Model(inputs = inputs, outputs = decoded)
        ensemble_outputs.append(model)

    return ensemble_outputs

encoder, inputs = create_encoder()
ensemble_models = create_ensemble(encoder, inputs)


In [ ]:
# Decoder Separated Training
'''
# Run the Second Model
EarlyStopping(monitor = 'loss', min_delta = 0, patience = 3,  mode = 'auto')
early_stopping = EarlyStopping()
#awgn_callback = AWGNCallback(SNR_DB)

# First Encoder-Decoder Training
first_decoder = ensemble_models[0]
first_decoder.compile(optimizer='adam', loss='mse')
first_decoder.fit(train_image, train_image, epochs = 30, batch_size = 32, callbacks=[early_stopping])
#first_decoder.fit(trainX, trainY, epochs=5, batch_size=32, callbacks=[early_stopping, awgn_callback])

# Encoder Freezing after first AE training
encoder.trainable = False
print("Encoder Training is over")

# Residual Decoder Training
for i in range(1, Decoder_count):
    model_i = ensemble_models[i]
    model_i.compile(optimizer='adam', loss='mse')
    model_i.fit(train_image, train_image, epochs  = 5, batch_size = 32, callbacks=[early_stopping])
    #model_i.fit(trainX, trainY, epochs=10, batch_size=32, callbacks=[early_stopping, awgn_callback])
    print("Decoder ", i+1, "th Training is over")
'''

In [ ]:
# Run the Second Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

best_val_losses = []
best_models = []

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, mode='auto')
# awgn_callback = AWGNCallback(SNR_DB)

# Training all Encoder-Decoder pairs
for i, model in enumerate(ensemble_models):
    model.compile(optimizer='adam', loss='mse')

    # Save the best model for the current encoder-decoder pair
    #model_checkpoint = ModelCheckpoint(f'best_model_{i}.h5', monitor='val_loss', save_best_only=True, verbose=1) 
    #history = model.fit(train_image, train_image, validation_split=0.2, epochs=1, batch_size=32, callbacks=[early_stopping, model_checkpoint])
    model.fit(train_image, train_image, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])
    # Get the best validation loss for the current model
    #best_val_loss = min(history.history['val_loss'])
    #best_val_losses.append(best_val_loss)
    
    print(f"Encoder-Decoder {i + 1}th Training is over")

# Find the index of the model with the lowest validation loss
#best_model_index = np.argmin(best_val_losses)
#print("Best Training Epoch is : ", best_model_index)

# Load the best model
#best_model = load_model(f'best_model_{best_model_index}.h5')

# Assign the best model to ensemble_models
#ensemble_models = best_model




In [ ]:
# Train Data Ensemble Verification

def ensemble_predict(testX, ensemble_models):
    predictions = []

    for model in ensemble_models:
        model_prediction = model.predict(testX)
        predictions.append(model_prediction)
        print("model_prediction : ",model_prediction.shape, model_prediction)
        print("prediction size : ",np.array(predictions).shape)
    ensemble_prediction = np.mean(predictions, axis=0)
    return ensemble_prediction

final_output = ensemble_predict(train_image, ensemble_models)

print("Ensemble prediction Average(Train):", final_output)
print("Train image : ",train_image)

In [ ]:
# Plotting
original_images = np.copy(train_image)
trainY_back_1 = original_images.reshape(-1, original_images.shape[-1])
trainY_back = HJ_backMinMax(trainY_back_1)

decoded_images_input = np.copy(train_image)
decoded_images = ensemble_predict(decoded_images_input,ensemble_models)
print("Original Decoded : ", decoded_images.shape)

decoded_images = np.squeeze(decoded_images, axis = 3)
trainY_predict_2 = decoded_images.reshape(-1, decoded_images.shape[-1])
trainY_predict_back = HJ_backMinMax(trainY_predict_2)
print("Decoded : ", trainY_predict_back.shape)


In [ ]:
print("Original : ", trainY_back)
print("Predicted : " ,trainY_predict_back)

In [ ]:
#Plotting Training data
import matplotlib.pyplot as plt

seq_length = 0
plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(trainY_back[seq_length:,0], color = 'blue')
plt.plot(trainY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(trainY_back[seq_length:,1], color = 'blue')
plt.plot(trainY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(trainY_back[seq_length:,2], color = 'blue')
plt.plot(trainY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(trainY_back[seq_length:,3], color = 'blue')
plt.plot(trainY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])

error_current_1 = trainY_back[:,0] - trainY_predict_back[:,0]
error_temp_1 = trainY_back[:,1] - trainY_predict_back[:,1]
error_soc_1 = trainY_back[:,2] - trainY_predict_back[:,2]
error_voltage_1 = trainY_back[:,3] - trainY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])
plt.ylim(-10, 10)

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-2, 2)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-10, 10)


In [ ]:
# Add the Noise
test_image2 = np.copy(test_image)
test_image_AWGN = add_awgn_noise(test_image2, snr_dB)

test_final_output = ensemble_predict(test_image, ensemble_models)
print("Final Output : ", test_final_output.shape)

In [ ]:
# Testing
testY_predict = np.copy(test_final_output)

print(testY_predict.shape)
testY_predict_1 = np.squeeze(testY_predict, axis = 3)
testY_predict_2 = testY_predict_1.reshape(-1, testY_predict_1.shape[-1])
testY_predict_back = HJ_backMinMax(testY_predict_2)

test_image_input = np.copy(test_image)
testY_back_1 = test_image_input.reshape(-1, test_image_input.shape[-1])
testY_back = HJ_backMinMax(testY_back_1)

print(testY_predict_back.shape)
print(testY_back.shape)

In [ ]:
#Plotting Test Data
import matplotlib.pyplot as plt

seq_length = 0

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(testY_back[seq_length:,0], color = 'blue')
plt.plot(testY_predict_back[:,0], color = 'red')
plt.legend(['Real Current','Predicted Current'])

plt.subplot(2,2,2)
plt.plot(testY_back[seq_length:,1], color = 'blue')
plt.plot(testY_predict_back[:,1], color = 'red')
plt.legend(['Real Temperature','Predicted Temperature'])

plt.subplot(2,2,3)
plt.plot(testY_back[seq_length:,2], color = 'blue')
plt.plot(testY_predict_back[:,2], color = 'red')
plt.legend(['Real SOC','Predicted SOC'])

plt.subplot(2,2,4)
plt.plot(testY_back[seq_length:,3], color = 'blue')
plt.plot(testY_predict_back[:,3], color = 'red')
plt.legend(['Real Voltage','Predicted Voltage'])


error_current_1 = testY_back[:,0] - testY_predict_back[:,0]
error_temp_1 = testY_back[:,1] - testY_predict_back[:,1]
error_soc_1 = testY_back[:,2] - testY_predict_back[:,2]
error_voltage_1 = testY_back[:,3] - testY_predict_back[:,3]

plt.figure(figsize = (10,5))
plt.subplot(2,2,1)
plt.plot(error_current_1, color = 'green')
plt.legend(['Current Error'])
plt.ylim(-5, 5)

plt.subplot(2,2,2)
plt.plot(error_temp_1, color = 'green')
plt.legend(['Temperature Error'])
plt.ylim(-1, 1)

plt.subplot(2,2,3)
plt.plot(error_soc_1, color = 'green')
plt.legend(['SOC Error'])
plt.ylim(-1, 1)

plt.subplot(2,2,4)
plt.plot(error_voltage_1, color = 'green')
plt.legend(['Voltage Error'])
plt.ylim(-5, 5)